# Change Schedule



Prerequisite :
1. A deployed prompt with monitoring enabled.(https://dataplatform.cloud.ibm.com/docs/content/wsj/model/wos-eval-prompt-spaces.html?context=wx&audience=wdp).
2. Your IBM Cloud API Key
3. The space id where your prompt is deployed.You can find it at Deployments > YOUR_SPACE > Manage

In [1]:
#install the required libraries
!pip install ibm-watson-openscale

In [2]:
#set the required variables
CLOUD_API_KEY = "YOUR_CLOUD_API_KEY"
SPACE_ID = "DEPLOYMENT_SPACE_ID"

In [3]:
#setup the openscale client
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
client = APIClient(authenticator=authenticator)
print(client.version)

3.0.39


In [4]:
#list available datamarts
client.data_marts.show()

WOS_AUTO_CREATED,This data-mart was automatically created by Watson X.Gov on behalf of users for prompt evaluations.,True,active,2024-07-23 10:17:07.445000+00:00,03881cc5-d985-4eb5-942b-652679599eb1


### Select the data mart where your monitoring is happenning. You can get this information from the Evaluations Tab > Actions > View Model information

In [5]:
DATA_MART_ID = "03881cc5-d985-4eb5-942b-652679599eb1"

In [6]:
#list subscriptions for the given deployment space
client.subscriptions.show(space_id=SPACE_ID)

550aadc7-2cb7-4ff7-aff2-5c0adb2233b6,prompt,Query Classification,03881cc5-d985-4eb5-942b-652679599eb1,60e9b545-6557-4afd-97c6-9f49331ec131,Query Classification,fa1aad58-dc20-453d-8ad6-fa6799ce969d,active,2024-08-08 12:21:17.851000+00:00,a3742fc1-ca91-4127-ba6a-f38e567c394c
95896fa1-4fd3-4756-8d46-a3629e990863,prompt,SOP RAG,03881cc5-d985-4eb5-942b-652679599eb1,f78653df-cbcc-480c-affd-1d7f50ed29c1,SOP RAG,fa1aad58-dc20-453d-8ad6-fa6799ce969d,active,2024-08-06 10:03:20.903000+00:00,eb4352f2-a8ac-4377-a206-e4dcd87ae49c
d245f7b9-426b-4119-b972-814f28be9468,prompt,IssueType Identification,03881cc5-d985-4eb5-942b-652679599eb1,ee290866-249a-486b-9c9a-ed170e5fb20c,IssueType Identification,fa1aad58-dc20-453d-8ad6-fa6799ce969d,active,2024-08-07 08:09:11.060000+00:00,ba02c916-90e8-43a8-a7df-1c150c166749
b41769e3-4beb-436e-9319-0ef22c13b5ae,prompt,Query Modification,03881cc5-d985-4eb5-942b-652679599eb1,2a3d2461-5043-4001-97de-fc4419af3bcf,Query Modification,fa1aad58-dc20-453d-8ad6-fa6799ce969d,active,2024-08-06 04:51:52.595000+00:00,5c26ad72-efe8-4836-8ff8-ec68d8b809f5
f9b1001b-ddba-4f54-9fa5-37277fc957f7,prompt,Summarization,03881cc5-d985-4eb5-942b-652679599eb1,deb76bbb-1f6d-4970-8644-150431210aa3,Summarization,fa1aad58-dc20-453d-8ad6-fa6799ce969d,active,2024-08-05 09:34:27.151000+00:00,96f2a3dc-70e8-4d1b-8a99-c6e8df00890a


### Select the subscription where your monitoring is happenning. You can also get this information from the Evaluations Tab > Actions > View Model information

In [7]:
SUBSCRIPTION_ID = "96f2a3dc-70e8-4d1b-8a99-c6e8df00890a"

In [8]:
#get list of monitor instances available for your subscription
monitor_instances_info = client.monitor_instances.list(
        data_mart_id=DATA_MART_ID,
        target_target_id= SUBSCRIPTION_ID,
        target_target_type="subscription"
     )
print(monitor_instances_info.result.monitor_instances)

[<ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f89283e69e0>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f89283e72e0>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f89283e74f0>, <ibm_watson_openscale.base_classes.watson_open_scale_v2.MonitorInstanceResponse object at 0x7f89280906d0>]


In [9]:
#list different monitor definitions available for your subscription
for instance in monitor_instances_info.result.monitor_instances:
    print(instance.entity.monitor_definition_id)

drift_v2
model_health
mrm
generative_ai_quality


In [10]:
def get_monitor_instance_details(monitor_definition):
    for instance in monitor_instances_info.result.monitor_instances:
        if instance.entity.monitor_definition_id == monitor_definition :
            return instance
    return -1

In [11]:
#get details of given monitor definitions. Pass any name listed in the above definition list 
instance_details = get_monitor_instance_details("generative_ai_quality")
instance_id = instance_details.metadata.id
instance_id

'786cad29-db33-4183-8d52-3fa4f807be6a'

In [13]:
payload = [{
  "op": "replace",
  "path": "/schedule",
  "value": {
    "repeat_interval": 60,
    "repeat_type": "minute",
    "repeat_unit": "minute",
    "status": "enabled"
  }
}]

print(client.monitor_instances.update(monitor_instance_id=instance_id, patch_document=payload).result)



{
  "metadata": {
    "id": "786cad29-db33-4183-8d52-3fa4f807be6a",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/6c26e1f11ef745238913798de41a0653:03881cc5-d985-4eb5-942b-652679599eb1:monitor_instance:786cad29-db33-4183-8d52-3fa4f807be6a",
    "url": "/v2/monitor_instances/786cad29-db33-4183-8d52-3fa4f807be6a",
    "created_at": "2024-08-05T09:34:32.165000Z",
    "created_by": "iam-ServiceId-b317a8da-d926-496e-b0ca-6bcc57f556ae",
    "modified_at": "2024-08-13T11:28:18.920000Z",
    "modified_by": "IBMid-6660010MZ1"
  },
  "entity": {
    "data_mart_id": "03881cc5-d985-4eb5-942b-652679599eb1",
    "monitor_definition_id": "generative_ai_quality",
    "target": {
      "target_type": "subscription",
      "target_id": "96f2a3dc-70e8-4d1b-8a99-c6e8df00890a"
    },
    "parameters": {
      "last_processed_ts": "2024-08-12T19:36:39.957241Z",
      "metrics_configuration": {
        "hap_input_score": {},
        "sentence_similarity": {},
        "text_quality": {},
        "ro